In [1]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# #Transforms
# transformer=transforms.Compose([
#     transforms.Resize((150,150)),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
#     transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
#                         [0.5,0.5,0.5])
# ])




# Define transformations for data augmentation and normalization
transformer = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomVerticalFlip(),    # Randomly flip the image vertically
    transforms.RandomRotation(degrees=30),  # Randomly rotate the image up to 30 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Randomly change brightness, contrast, saturation, and hue
    transforms.RandomResizedCrop(size=(150, 150), scale=(0.8, 1.0), ratio=(0.75, 1.333)),  # Randomly crop a portion of the image and resize it
    transforms.RandomGrayscale(p=0.1),  # Randomly convert the image to grayscale with probability 0.1
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # Randomly apply perspective transformation
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize the image with mean and standard deviation
])




In [4]:
#Dataloader

#Path for training and testing directory
train_path=r"C:\Users\rohin\Desktop\New folder (2)\biomet_pattern_and_people_detection\Task 2 DeepFakes Detection-20240228\cropped_cannyApplication\kev_data_aug\train"
test_path=r"C:\Users\rohin\Desktop\New folder (2)\biomet_pattern_and_people_detection\Task 2 DeepFakes Detection-20240228\cropped_cannyApplication\kev_data_aug\test"

# DataLoader with data augmentation
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform=transformer),
    batch_size=8, shuffle=True
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform=transformer),
    batch_size=8, shuffle=True
)

In [5]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)


['fake', 'real']


In [6]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        #self.fc=nn.Linear(in_features=200 * 200 * 32,out_features=num_classes)
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            





In [7]:
model=ConvNet(num_classes=2).to(device)

In [8]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [10]:
num_epochs=20
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,test_count)

1829 424


In [11]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'RGB_FE_01.model')
        best_accuracy=test_accuracy
    

Epoch: 0 Train Loss: tensor(4.8637) Train Accuracy: 0.74412247129579 Test Accuracy: 0.8207547169811321
Epoch: 1 Train Loss: tensor(1.0995) Train Accuracy: 0.7873154729360307 Test Accuracy: 0.8466981132075472
Epoch: 2 Train Loss: tensor(0.4536) Train Accuracy: 0.8190267905959541 Test Accuracy: 0.8726415094339622
Epoch: 3 Train Loss: tensor(0.3357) Train Accuracy: 0.8671405139420448 Test Accuracy: 0.7877358490566038
Epoch: 4 Train Loss: tensor(0.3305) Train Accuracy: 0.847457627118644 Test Accuracy: 0.875
Epoch: 5 Train Loss: tensor(0.3305) Train Accuracy: 0.8578458173865501 Test Accuracy: 0.8584905660377359
Epoch: 6 Train Loss: tensor(0.3212) Train Accuracy: 0.8633132859486058 Test Accuracy: 0.9080188679245284
Epoch: 7 Train Loss: tensor(0.3324) Train Accuracy: 0.8578458173865501 Test Accuracy: 0.8349056603773585
Epoch: 8 Train Loss: tensor(0.3044) Train Accuracy: 0.8665937670858392 Test Accuracy: 0.8820754716981132
Epoch: 9 Train Loss: tensor(0.2967) Train Accuracy: 0.8715144887916895 